<a href="https://colab.research.google.com/github/zzhenxi/study-NLP-with-PyTorch/blob/main/%5BChapter3%5D_%EB%A0%88%EC%8A%A4%ED%86%A0%EB%9E%91_%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%84%B1_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/my_dataset/yelp_review/my_reviews_with_splits_lite.csv')

In [3]:
df.head()

,rating,review,split
0,negative,terrible place to work for i just heard a stor...,train
1,negative,"hours , minutes total time for an extremely s...",train
2,negative,my less than stellar review is for service . w...,train
3,negative,i m granting one star because there s no way t...,train
4,negative,the food here is mediocre at best . i went aft...,train


# 데이터 백터화 클래스

In [ ]:
from torch.utils.data import Dataset

class ReviewDataset(Dataset):
  def __init__(self, review_df, vectorizer):
    self.review_df = review_df
    self._vecrotizer = vectorizer

    self.train_df = self.review_df[self.review_df.split =='train']
    self.train_size = len(self.train_df)

    self.val_df = self.review_df[self.review_df.split =='val']
    self.validation_size = len(self.val_df)

    self.test_df = self.review_df[self.review.split]